# Convert BGR images to grayscale

`imageio` saves images to BGR png.  These needs to be converted to grayscale to be compatible with current pipeline.

In [1]:
! rsync -a /kaggle/input/mmdetection-v280/mmdetection /
! pip install /kaggle/input/mmdetection-v280/src/mmpycocotools-12.0.3/mmpycocotools-12.0.3/
! pip install /kaggle/input/hpapytorchzoo/pytorch_zoo-master/
! pip install /kaggle/input/hpacellsegmentation/HPA-Cell-Segmentation/
! pip install /kaggle/input/iterative-stratification/iterative-stratification-master/

! cp -r /kaggle/input/kgl-humanprotein-data/kgl_humanprotein_data /
! cp -r /kaggle/input/humanpro/kgl_humanprotein /

import sys
sys.path.append('/kgl_humanprotein/')

Processing /kaggle/input/mmdetection-v280/src/mmpycocotools-12.0.3/mmpycocotools-12.0.3
  Created wheel for mmpycocotools: filename=mmpycocotools-12.0.3-cp37-cp37m-linux_x86_64.whl size=272906 sha256=0c4f51384bee4a8c0a6f7cb604726e20ecbec4ae992705e2b4257c33beb0bad0
  Stored in directory: /root/.cache/pip/wheels/80/e0/da/3288fdf3965b5c9090f368462db9d28be2c82013f51821090a
Successfully built mmpycocotools
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Processing /kaggle/input/hpapytorchzoo/pytorch_zoo-master
  Created wheel for pytorch-zoo: filename=pytorch_zoo-0.0.0-py3-none-any.whl size=30139 sha256=a905832615b74e9ba81055428f19bcbdfcf96d7026305e26b912caf804b06f1c
  Stored in directory: /root/.cache/pip/wheels/7f/18/21/aff5a8914e22461b2b025a9629c2b70464c36183caaf12bc09
Successfully built pytorch-zoo
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Processing /kaggle/input/hpacells

In [2]:
import os
import time
from pathlib import Path
import shutil
import zipfile
import functools
import multiprocessing
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import KFold,StratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import torch
from torch.backends import cudnn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn import DataParallel
import matplotlib.pyplot as plt
from tqdm import tqdm

from kgl_humanprotein.utils.common_util import *
from kgl_humanprotein.config.config import *
from kgl_humanprotein.data_process import *
from kgl_humanprotein.datasets.tool import image_to_tensor
from kgl_humanprotein.networks.imageclsnet import init_network
from kgl_humanprotein.layers.loss import *
from kgl_humanprotein.layers.scheduler import *
from kgl_humanprotein.utils.augment_util import train_multi_augment2
from kgl_humanprotein.utils.log_util import Logger
from kgl_humanprotein.run.train import *

run on 0d71ef66b3f4


In [3]:
%cd /kaggle

/kaggle


In [4]:
%%time

dir_img = Path('/kaggle/input/humanpro-publichpa-1/publichpa/publichpa/')

dir_grey = Path('/kaggle/publichpa')
dir_grey.mkdir(exist_ok=True)

fns = dir_img.ls()

def ibgr2gray(i):
    n = fns[i]
    img = cv2.imread(str(n), cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    n_out = dir_grey/n.name
    cv2.imwrite(str(n_out), img)
    return n, n_out
    
    
CPU_COUNT = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=CPU_COUNT)
for n, n_out in pool.imap(ibgr2gray, range(len(fns))):
    print(f'{str(n):80s} -----> {str(n_out):60s}')

/kaggle/input/humanpro-publichpa-1/publichpa/publichpa/804_H8_2_blue.png         -----> /kaggle/publichpa/804_H8_2_blue.png                         
/kaggle/input/humanpro-publichpa-1/publichpa/publichpa/290_C12_1_yellow.png      -----> /kaggle/publichpa/290_C12_1_yellow.png                      
/kaggle/input/humanpro-publichpa-1/publichpa/publichpa/69_A6_2_red.png           -----> /kaggle/publichpa/69_A6_2_red.png                           
/kaggle/input/humanpro-publichpa-1/publichpa/publichpa/1786_A5_9_red.png         -----> /kaggle/publichpa/1786_A5_9_red.png                         
/kaggle/input/humanpro-publichpa-1/publichpa/publichpa/579_F2_2_red.png          -----> /kaggle/publichpa/579_F2_2_red.png                          
/kaggle/input/humanpro-publichpa-1/publichpa/publichpa/1766_A1_5_yellow.png      -----> /kaggle/publichpa/1766_A1_5_yellow.png                      
/kaggle/input/humanpro-publichpa-1/publichpa/publichpa/1784_A4_4_blue.png        -----> /kaggle/publichpa/

In [5]:
%cd /kaggle
! zip -rq publichpa.zip publichpa/
! cp publichpa.zip /kaggle/working/.
%cd /kaggle

/kaggle
/kaggle
